# Evaluation TextGrad with TextualVerifier in Loss

In [1]:
import re
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

import textgrad as tg
from textgrad.engine import get_engine
from textgrad.variable import Variable
from textgrad.loss import TextLoss
from textgrad.optimizer import TextualGradientDescent
from textgrad.verifier import TextualVerifierWithTracker


## Load Dataset

In [2]:
initial_solution = pd.read_csv("datasets/initial-solution.csv")
initial_solution

,id,formatted_question,raw_solution,correct_answer,source,subject
0,2,Answer the following multiple choice question....,Here's how we can determine the number of carb...,A,GPQA-Diamond,-
1,4,Answer the following multiple choice question....,Maxwell's equations in our universe are:\n\n1....,A,GPQA-Diamond,-
2,8,Answer the following multiple choice question....,Here's how we can analyze the results and dete...,B,GPQA-Diamond,-
3,1,Answer the following multiple choice question....,The energy-time uncertainty principle states t...,A,GPQA-Diamond,-
4,22,Answer the following multiple choice question....,The question asks about the oxidizing power of...,D,GPQA-Diamond,-
...,...,...,...,...,...,...
407,394,Answer the following multiple choice question....,The police car is moving towards the wall. Le...,B,MMLU-CP,college_physics
408,384,Answer the following multiple choice question....,Here's how we can solve this problem:\n\n1. **...,A,MMLU-CP,college_physics
409,404,Answer the following multiple choice question....,The diffraction of electrons by a crystal latt...,A,MMLU-CP,college_physics
410,390,Answer the following multiple choice question....,Here's how we can solve this problem:\n\n1. **...,D,MMLU-CP,college_physics


## Setup Engine

In [3]:
engine = get_engine("gemini-1.5-pro")
tg.set_backward_engine("gemini-1.5-pro", override=True)

/Users/eugeniusms/Development/SKRIPSI/sevet/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Experiment Iterations

In [ ]:
def extract_answer(text):
    # Now extract correctly
    match = re.search(r"(?i)Answer\s*:\s*([A-D])", text, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    return "Z" # Z -> means for None (Incorrect Answer)

In [4]:
def evaluate(row_data, engine):
    try:
        match = initial_solution[initial_solution["id"] == row_data["id"]]
        if match.empty:
            return None  # or raise error
        formatted_question = match.iloc[0]["formatted_question"]
        
        solution = Variable(row_data["raw_solution"],
                            requires_grad=True,
                            role_description=f"Solution to the math question: {formatted_question}")
        loss_system_prompt = Variable("""You will evaluate a solution to a math question. 
                                        Do not attempt to solve it yourself, do not give a solution, 
                                        only identify errors. Be super concise.""",
                                        requires_grad=False,
                                        role_description="system prompt")
        verification_task_prompts = [
            """
            1. When using $instruction to $instance, I got the loss $calculation.
            2. Evaluate the loss $calculation value correctly reflects the performance of the instance.
            3. If the loss $calculation is incorrect or inconsistent, provide the corrected version of the loss $calculation. 
            4. Do NOT calculate the solution/instance, evaluate $calculation ONLY.
            """
        ]

        optimizer = TextualGradientDescent([solution])
        loss = TextLoss(loss_system_prompt, engine=engine)
        loss_value = loss(solution) # Forward method in Loss Function
        original_loss = loss_value.value

        # TextualVerifier
        verifier = TextualVerifierWithTracker(
            verifier_engine=engine, 
            use_cot_generation=False,
            use_step_breakdown=False,
            verification_task_prompts=verification_task_prompts,
            enable_logging=True
        )

        verified_result = verifier.verify(instance=solution,
                                        instruction=loss_system_prompt,
                                        calculation=loss_value)

        tracker_data = verifier.get_tracker() # Tracker data     

        loss_value.set_value(verified_result.value) 
        
        loss_value.backward()
        optimizer.step()

        return {
            "id": row_data["id"],
            "correct_answer": row_data["correct_answer"],
            "original_solution": row_data["raw_solution"],
            "original_loss": original_loss,
            "verified_loss": verified_result,
            "verified_solution": solution.value,
            "verified_solution_answer": extract_answer(solution.value), 
            "source": row_data["source"],
            "subject": row_data["subject"],
            "success": True,
            "error_message": None,
            "processing_time_ms": tracker_data['processing_time_ms'],
            "total_llm_calls": tracker_data['total_llm_calls'],
            "total_input_tokens": tracker_data['total_input_tokens'],
            "total_output_tokens": tracker_data['total_output_tokens'],
            "result": tracker_data
        }
        
    except Exception as e:
        # Return error information
        return {
            "id": row_data["id"],
            "correct_answer": "",
            "original_solution": "",
            "original_loss": "",
            "verified_loss": "",
            "verified_solution": "",
            "source": "",
            "subject": "",
            "success": False,
            "error_message": str(e),
            "processing_time_ms": 0,
            "total_llm_calls": 0,
            "total_input_tokens": 0,
            "total_output_tokens": 0,
            "result": {}
        }

In [5]:
def run_evaluation():
    results = []
    start_time = time.time()
    
    with ThreadPoolExecutor(max_workers=32) as executor:
        futures = [
            executor.submit(evaluate, row.to_dict(), engine) 
            for _, row in initial_solution[400:401].iterrows()  
        ]
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
            try:
                result = future.result(timeout=None)
                if result is not None:
                    results.append(result)
            except Exception as e:
                print(f"Task failed: {e}")
                error_result = {
                    'success': False,
                    'error_message': str(e),
                    'result': {}
                }
                results.append(error_result)

    experiment_df = pd.DataFrame(results)
    print(f"Completed in {time.time() - start_time:.1f} seconds")
    experiment_df.to_csv('results/result-03-textgrad-tv-l.csv', index=False)
    
    return experiment_df

In [6]:
run_evaluation()

Processing: 100%|██████████| 1/1 [00:00<00:00,  9.84it/s]

INFO:textgrad:TextualVerifier: Start verification process...
INFO:textgrad:TextualVerifier: Ready to verify 1 calculation steps...
INFO:textgrad:TextualVerifier: Verifying step 1/1...
INFO:textgrad:TextualVerifier: Generating step 1 variant 1/1...
INFO:textgrad:TextualVerifier: Running majority voting for step 1...
["Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                        Do not attempt to solve it yourself, do not give a solution, \n                                        only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> The object is thrown horizontally, so the initial vertical velocity is 0 m/s. The object is subject to gravity, which causes it to accelerate downwards at 9.8 m/s². We are given that the time of flight is 2.0 s. We can use the following kinematic equation to find the vertical displacement, which corresponds to the height from which the object was thro

,id,correct_answer,original_solution,original_loss,verified_loss,verified_solution,source,subject,success,error_message,processing_time_ms,total_llm_calls,total_input_tokens,total_output_tokens,result
0,406,B,"The object is thrown horizontally, so the init...",The solution correctly identifies the initial ...,<VERIFIED>```The calculation correctly identif...,"The object is thrown horizontally, so the init...",MMLU-CP,college_physics,True,None,41.615967,2,553,207,{'setup': {'verifier_engine': '<textgrad.engin...
